<a href="https://colab.research.google.com/github/nikodallanoce/XAI_PhD_prj/blob/master/Project_XAI_DS4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/nikodallanoce/XAI_PhD_prj.git
! git clone https://github.com/pellungrobe/XAI_course_2024.git

Cloning into 'XAI_PhD_prj'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 29 (delta 2), reused 8 (delta 2), pack-reused 21
Receiving objects: 100% (29/29), 157.62 MiB | 23.99 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (8/8), done.
Cloning into 'XAI_course_2024'...
remote: Enumerating objects: 4098, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 4098 (delta 30), reused 36 (delta 26), pack-reused 4050
Receiving objects: 100% (4098/4098), 1006.87 MiB | 26.11 MiB/s, done.
Resolving deltas: 100% (450/450), done.
Updating files: 100% (3055/3055), done.


In [2]:
! pip install lightning > /dev/null

In [3]:
! gdown --fuzzy https://drive.google.com/file/d/1cgEKSkQ-LQX4VYa-oRliY0W6B3XsQKAi/view?usp=sharing
! 7z x "/content/New Plant Diseases Dataset(Augmented).zip" #contains only validation and test set.

Downloading...
From (original): https://drive.google.com/uc?id=1cgEKSkQ-LQX4VYa-oRliY0W6B3XsQKAi
From (redirected): https://drive.google.com/uc?id=1cgEKSkQ-LQX4VYa-oRliY0W6B3XsQKAi&confirm=t&uuid=23a75ed8-dcf3-4ca7-88b7-d1252ee37940
To: /content/New Plant Diseases Dataset(Augmented).zip
100% 289M/289M [00:01<00:00, 236MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 289378688 bytes (276 MiB)

Extracting archive: /content/New Plant Diseases Dataset(Augmented).zip
 23% 4096 Open              --
Path = /content/New Plant Diseases Dataset(Augmented).zip
Type = zip
Physical Size = 289378688

  0%      2% 577 - New Plant Diseases Dataset(Au . 14___JR_FrgE.S 8627_270deg.JPG

In [4]:
from XAI_PhD_prj.CustomEfficientNet import CustomEfficientNet
import lightning.pytorch as pl
from tqdm.auto import tqdm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

torch.set_float32_matmul_precision('high')

In [5]:
model = CustomEfficientNet.load_from_checkpoint("/content/XAI_PhD_prj/checkpoints/efficient_net_scratch-v15.ckpt")
model.eval()
model.freeze()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [6]:
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os

data_dir = "New Plant Diseases Dataset(Augmented)"
valid_dir = os.path.join(data_dir, "valid")
batch_size = 32
valid = ImageFolder(valid_dir, transform=transforms.ToTensor())
valid_dl = DataLoader(valid, batch_size, num_workers=4, pin_memory=True, persistent_workers=True)
test_dir = os.path.join(data_dir, "test")
test = ImageFolder(test_dir, transform=transforms.ToTensor())
for k in test.class_to_idx:
    test.class_to_idx[k] = valid.class_to_idx[k]
test.samples = test.make_dataset(test_dir, test.class_to_idx,
                                 extensions=(
                                     ".jpg", ".jpeg", ".png", ".ppm", ".bmp", ".pgm", ".tif", ".tiff", ".webp"))
test_dl = DataLoader(test, batch_size, num_workers=4, pin_memory=True, persistent_workers=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
miss_classified_images = []
from torchmetrics import Accuracy

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
acc = Accuracy("multiclass", num_classes=len(valid.classes)).to(device)

with torch.no_grad():
    for image_lbl in tqdm(valid_dl):
        image, label = image_lbl
        image, label = image.to(device), label.to(device)
        probs = model(image)
        acc.update(probs, label)
        classes = torch.argmax(probs, dim=1)
        idx_differ = torch.where(classes == label, 0, 1).nonzero()
        for i in idx_differ:
            i = int(i)
            miss_classified_images.append({"img": image[i], "prediction": int(classes[i]), "label": int(label[i])})
        # if len(miss_classified_images) > 0:
        #     break

acc.compute()

  0%|          | 0/550 [00:00<?, ?it/s]

In [ ]:
from lightning.pytorch import Trainer
Trainer().test(model, test_dl)

In [ ]:
import matplotlib.pyplot as plt
import random

# 3) Select the image we want to give an explanation for
wrong_pred = random.choice(miss_classified_images)
image = wrong_pred['img'].to("cpu")
print(image.shape)
plt.imshow(image.permute(1,2,0).numpy())
plt.show()

In [ ]:
import sys
sys.path.insert(0, "XAI_course_2024")

## Intgrad

In [ ]:
from XAI_course_2024.xailib.explainers.intgrad_explainer import IntgradImageExplainer

model = model.to(device)
# 1) Create the Explainer
ig = IntgradImageExplainer(model)

# 2) Fit the Explainer
ig.fit()

#MNIST_x_data_train[80:81]


# 4a) Preprocessing function: function that takes as input an image and return the correct format for the black box
def preprocessing(image):
    return torch.from_numpy(image).float() if not isinstance(image, torch.Tensor) else image


# 4b) Predict Function of our black box: It needs to take as input the output of preprocessing and return an array of probabilities of classes with shape (-1,num_classes)
def predict(image):
    return torch.nn.functional.softmax(model(image.unsqueeze(dim=0).to(device)), dim=1)
    #return MNIST_CNN(image.unsqueeze(0))


# 5) Explain an Instance
intgrad_pt_score = ig.explain(image, wrong_pred["label"], 'half', preprocessing=preprocessing, predict=predict,
                              model_type='pytorch', cuda=device=="cuda")

plt.imshow(intgrad_pt_score, cmap='coolwarm')
plt.colorbar()
plt.axis('off');

In [ ]:
from PIL import Image
img = Image.fromarray(intgrad_pt_score, "RGB")
plt.imshow(img)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
np_image = image.numpy() if not isinstance(image, np.ndarray) else image

ax[0].imshow(preprocessing(np_image).permute(1, 2, 0).numpy())
ax[1].imshow(np.sum(intgrad_pt_score, axis=-1), cmap='coolwarm')
ax[2].imshow(preprocessing(np_image).permute(1, 2, 0).numpy(), alpha=1)
ax[2].imshow(np.sum(intgrad_pt_score, axis=-1), cmap='coolwarm', alpha=0.8);

## LIME

In [ ]:
!pip install lime

In [ ]:
from XAI_course_2024.xailib.explainers.lime_explainer import LimeXAIImageExplainer
from skimage.color import label2rgb, gray2rgb, rgb2gray

# 1) Create the Explainer
model = model.to(device)
lm = LimeXAIImageExplainer(model)

# 2a) Fit the Explainer
lm.fit()


# 2b) Create custom classifier function if not predict
# function that takes as input an array of images (the LIME neighbourhood) and return an array of (num_images, num_classes)
# If None will use black_box.predict function
def classifier_fn(images):
    with torch.no_grad():
        #images = torch.tensor(images/255).permute(0,3,1,2).float()
        images = torch.from_numpy(images).permute(0, 3, 1, 2).float()
        pred = torch.nn.functional.softmax(model(images.to(device)), dim=1).cpu().detach().numpy()
        return pred


In [ ]:
# 3) Explain an Instance
explanation = lm.explain((preprocessing(np_image).permute(1, 2, 0).numpy() * 255).astype(int), classifier_fn,
                         num_samples=1000)

# 4) Plot the results
lm.plot_lime_values(preprocessing(np_image).permute(1, 2, 0).numpy(), explanation)
lime_score = explanation.heatmap

## RISE

In [ ]:
from XAI_course_2024.xailib.explainers.rise_explainer import RiseXAIImageExplainer


class Rise_model_wrapper():
    def __init__(self, bb, input_size):
        self.model = bb
        self.input_size = input_size

    def predict(self, X):
        with torch.no_grad():
            images = torch.tensor(X).permute(0, 3, 1, 2).float()
            return torch.nn.functional.softmax(model(images.to(device)), dim=1).cpu().detach().numpy()


model = model.to(device)
rise_model = Rise_model_wrapper(model, (np_image.shape[1], np_image.shape[1]))
rise = RiseXAIImageExplainer(rise_model)

N = 1000  # number of random masks
s = 10  # cell_size = input_shape / s
p1 = 0.5  # masking probability
pred = wrong_pred['prediction']

rise.fit(N, s, p1)

sal = rise.explain(preprocessing(np_image).permute(1, 2, 0).cpu().detach().numpy())
rise_score = sal[pred, :]

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(preprocessing(np_image).permute(1, 2, 0).cpu().detach().numpy(), cmap='gray')
ax[0].axis('off')

ax[1].imshow(sal[pred, :], cmap='jet')
ax[1].axis('off')

ax[2].imshow(preprocessing(np_image).permute(1, 2, 0).cpu().detach().numpy(), cmap='gray')
ax[2].imshow(sal[pred, :], cmap='jet', alpha=0.5)
ax[2].axis('off');

## GradCAM

In [ ]:
! pip install ttach

In [ ]:
from XAI_course_2024.xailib.explainers.gradcam_explainer import GradCAMImageExplainer

model = CustomEfficientNet.load_from_checkpoint("/content/XAI_PhD_prj/checkpoints/efficient_net_scratch-v15.ckpt")
model = model.to(device)

explainer = GradCAMImageExplainer(model)

explainer.fit([model.model.features[2][0].block[1][0]])  #-5,36

attr = explainer.explain(preprocessing(np_image).to(device).unsqueeze(0), pred, model_type='pytorch')
gradcam_pt_score = attr[0, :]

fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].imshow(preprocessing(np_image).permute(1, 2, 0).cpu().detach().numpy(), cmap='gray')
ax[0].axis('off')

ax[1].imshow(attr[0, :], cmap='jet')
ax[1].axis('off')

ax[2].imshow(preprocessing(np_image).permute(1, 2, 0).cpu().detach().numpy(), cmap='gray')
ax[2].imshow(attr[0, :], cmap='jet', alpha=0.5)
ax[2].axis('off');

## GradCAM plus

In [ ]:
from XAI_course_2024.xailib.explainers.gradcam_explainer import GradCAMPlusPlusImageExplainer

explainer = GradCAMPlusPlusImageExplainer(model)

explainer.fit([model.model.features[2][0].block[1][0]])

attr = explainer.explain(preprocessing(image).to(device).unsqueeze(0), pred, model_type='pytorch')
gradcampp_pt_score = attr[0,:]

fig, ax = plt.subplots(1,3,figsize=(10,5))

ax[0].imshow(preprocessing(image).permute(1,2,0).cpu().detach().numpy(),cmap='gray')
ax[0].axis('off')

ax[1].imshow(attr[0,:],cmap='jet')
ax[1].axis('off')

ax[2].imshow(preprocessing(image).permute(1,2,0).cpu().detach().numpy(),cmap='gray')
ax[2].imshow(attr[0,:],cmap='jet',alpha=0.5)
ax[2].axis('off');

In [ ]:
methods = [np.sum(intgrad_pt_score,axis=-1),lime_score,rise_score,gradcam_pt_score,gradcampp_pt_score]
F, ax = plt.subplots(1,5,figsize=(20,5))
for i in range(5):
    ax[i].axis(False)
    ax[i].imshow(methods[i],cmap='coolwarm')

# XAI analysis

In [ ]:
from XAI_course_2024.xailib.metrics.insertiondeletion import ImageInsDel

def predict(image):
    with torch.no_grad():
        pred = torch.nn.functional.softmax(model.forward(torch.tensor(image).to(device)),dim=1)
        return pred

step = 256
mode = 'del'
metric = ImageInsDel(predict, mode, step, torch.zeros_like)

In [ ]:
from sklearn.metrics import auc
y = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, intgrad_pt_score, rgb=True)

x = np.arange(len(y))/(step*step)*step
x[-1] = 1.0
print('Score:', auc(x, y))

plt.plot(x, y, label=np.round(auc(x, y),4))
plt.fill_between(x, y, alpha=0.4)
plt.xlabel('Percentage of pixel removed')
plt.ylabel('Accuracy of the model')
plt.legend();

In [ ]:
from sklearn.metrics import auc

y_int = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, intgrad_pt_score, rgb=True)
y_rise = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, rise_score, rgb=True)
y_lime = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, lime_score, rgb=True)
y_gcam = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, gradcam_pt_score, rgb=True)
y_gcamp = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, gradcampp_pt_score, rgb=True)
x = np.arange(len(y_int))/(step*step)*step
x[-1] = 1.0

for name, y in zip(['intgrad','lime','rise','gcam','gcamp'],[y_int,y_lime,y_rise,y_gcam,y_gcamp]):
    plt.plot(x, y, label=f'{name}: {np.round(auc(x, y),4)}')
    plt.fill_between(x, y, alpha=0.4)
plt.xlabel('Percentage of pixel removed')
plt.ylabel('Accuracy of the model')
plt.legend();

In [ ]:
from sklearn.metrics import auc
[y, anim] = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, rise_score, rgb=True, verbose=1)

In [ ]:
anim

In [ ]:
import torch.nn as nn
from scipy.ndimage.filters import gaussian_filter
CH = 3
def gkern(klen, nsig):
    """Returns a Gaussian kernel array.
    Convolution with it results in image blurring."""
    # create nxn zeros
    inp = np.zeros((klen, klen))
    # set element at the middle to one, a dirac delta
    inp[klen//2, klen//2] = 1
    # gaussian-smooth the dirac, resulting in a gaussian filter mask
    k = gaussian_filter(inp, nsig)
    kern = np.zeros((CH, CH, klen, klen))
    for i in range(CH):
        kern[i, i] = k
    return torch.from_numpy(kern.astype('float32'))

# Function that blurs input image
def blur(image, klen=25, ksig=25):
    kern = gkern(klen, ksig)
    image = torch.tensor(image).float()
    return nn.functional.conv2d(image, kern, padding=klen//2)

F,ax = plt.subplots(1,2)
ax[0].imshow(preprocessing(np_image).permute(1,2,0).numpy(), cmap='gray')
ax[0].axis(False)
ax[1].imshow(blur(preprocessing(np_image)).permute(1,2,0),cmap='gray')
ax[1].axis(False);

In [ ]:
from XAI_course_2024.xailib.metrics.insertiondeletion import ImageInsDel

def predict(image):
    with torch.no_grad():
        pred = torch.nn.functional.softmax(model.forward(torch.tensor(image).to(device)),dim=1)
        return pred

step = 256
mode = 'ins'
metric = ImageInsDel(predict, mode, step, blur)

In [ ]:
from sklearn.metrics import auc

y_int = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, intgrad_pt_score, rgb=True)
y_rise = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, rise_score, rgb=True)
y_lime = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, lime_score, rgb=True)
y_gcam = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, gradcam_pt_score, rgb=True)
y_gcamp = metric(preprocessing(np_image).unsqueeze(0).numpy(), step, gradcampp_pt_score, rgb=True)
x = np.arange(len(y_int))/(256*256)*step
x[-1] = 1.0

for name, y in zip(['intgrad','lime','rise','gcam','gcamp'],[y_int,y_lime,y_rise,y_gcam,y_gcamp]):
    plt.plot(x, y, label=f'{name}: {np.round(auc(x, y),4)}')
    plt.fill_between(x, y, alpha=0.4)
plt.xlabel('Percentage of pixel inserted')
plt.ylabel('Accuracy of the model')
plt.legend();